# Método Simplex 

O método simplex é um procedimento inventado no metade final dos anos 1940 por George Dantzig para resolver programas lineares que funciona iterativamente. Cada iteração, dentro do espaço viável, melhora a função objetivo. É um algoritmo robusto que resolve o problema dual simultaneamente. A partir de um programa linear qualquer, o primeiro passo é transformar para a forma canônica e nesse formato, o problema é resolvido de maneira simples. 

## Forma canônica 

Um problema linear com a seguinte estrutura: 

1. As variáveis de decisão são não negativas; 
2. Todas as restrições são em forma de equações; 
3. Os coeficientes do lado direito das restrições são não negativos; 
4. Em cada restrição, uma variável de decisão é isolada com coeficiente +1, enquanto nas outras restrições, essa variável não aparece. Ela também aparece com coeficiente zero na função objetivo. 

## Critérios para solução do problema 

Consideremos inicialmente o problema em sua forma canônica. Temos três importantes critérios: 

1. Critério da ilimitação: estabelece quando o problema é ilimitado ou não; 
2. Critério da melhora: estabelece quando e como melhorar a função objetivo a partir de uma variável não básica; 
3. Critério da razão e pivotamento: estabelece como alterar a base de variáveis.

In [1]:
import numpy as np

Estamos interessados em maximizar uma função objetivo linear sujeita a retrições lineares de igualdade e desigualdade. Dessa forma, resolvemos problemas do tipo: 

$$\begin{align}
\max_x \ & c^T x \\
\mbox{tal que}  \ & A_{ub} x \leq b_{ub}, \\
& A_{eq} x = b_{eq}, \\
& l \leq x \leq u
\end{align}$$

de forma que $x$ é o vetor de variáveis de decisão, $c$, $b_{ub}, b_{eq}, l, u$ são vetores e $A_{ub}$ e $A_{eq}$ são matrizes também. 

In [215]:
class SimplexMethod: 
    
    def __init__(self, c, **kwargs): 
        
        c = c.reshape(-1,1)
        self.n_var = c.shape[0]
        A_ub = kwargs.get("A_ub", np.empty(shape=(0,self.n_var)))
        A_eq = kwargs.get("A_eq", np.empty(shape=(0,self.n_var)))
        b_ub = kwargs.get("b_ub", np.empty(shape=(0,1)))
        b_eq = kwargs.get("b_eq", np.empty(shape=(0,1)))
        # If bounds is an empty list, treating as [0, +inf].
        bounds = kwargs.get("bounds", [])
        
        self._check_dimensions(c, A_ub, A_eq, b_ub, b_eq)
        self.variables_names = ["x"+ str(i) for i in range(self.n_var)]
        
        c, A_ub, A_eq, b_ub, b_eq = self._bounds_handler(bounds, c, A_ub, A_eq, b_ub, b_eq)
        slack, A_ub, b_ub = self._add_slack_variables(A_ub, b_ub)
        
        # Joining all the equalities
        slack = np.vstack([slack, np.zeros(A_eq.shape[0], slack.shape[0])])
        A = np.vstack([A_ub, A_eq])
        b = np.vstack([b_ub, b_eq])
        slack, A, b = self._force_b_positive(slack, A, b)
        
        self.art_variables = self._add_artificial_variables(slack, A, c, b)
        
        
    def _check_dimensions(self, c, A_ub, A_eq, b_ub, b_eq): 
        
        if A_ub.shape[1] == self.n_var: 
            raise Exception("The number of columns of A_ub must be the number of lines of c.")
        elif A_eq.shape[1] == self.n_var:
            raise Exception("The number of columns of A_eq must be the number of lines of c.")
        elif b_ub.shape[0] == A_ub.shape[0]:
            raise Exception("The number of lines of A_ub must be the number of lines of b_ub.")
        elif b_eq.shape[0] == A_eq.shape[0]: 
            raise Exception("The number of lines of A_eq must be the number of lines of b_eq.")
            
    def _bounds_handler(self, bounds, c, A_ub, A_eq, b_ub, b_eq):
        
        l = np.array([b[0] for b in bounds], dtype = np.float64)
        u = np.array([b[1] for b in bounds], dtype = np.float64)
        if len(bounds) == 0: 
            return (A_ub, b_ub)
        elif len(bounds) < self.n_var: 
            raise Exception("You need to specify dimension of c bounds.")
        else:
            col_bar = self.n_var + sum(l == -np.inf)
            lin_bar = A_ub.shape[0] + sum(l > -np.inf) + sum(u < np.inf)
            A_ub_bar = np.zeros((lin_bar, col_bar))
            A_eq_bar = np.zeros((A_eq.shape[0], col_bar))
            b_ub_bar = np.zeros((lin_bar, 1))
            c_bar = np.zeros((col_bar, 1))
            
            col = 0
            lin = 0
            for i in range(self.n_var):
                if l[i] == -np.inf: 
                    A_ub_bar[:A_ub.shape[0], col] = A_ub[:,i]
                    A_ub_bar[:A_ub.shape[0], col+1] = -A_ub[:,i]
                    A_eq_bar[:A_eq.shape[0], col] = A_eq[:,i]
                    A_eq_bar[:A_eq.shape[0], col+] = -A_eq[:,i]
                    c_bar[col,1] = c[col]
                    c_bar[col,1] = -c[col]
                    if u[i] < np.inf: 
                        A_ub_var[A_ub.shape[0]+lin, col] = 1
                        A_ub_bar[A_ub.shape[0]+lin, col+1] = -1
                        b_ub_bar[A_ub.shape[0]+lin, 1] = u[i]
                        lin += 1
                    old_name = self.variables_names[i]
                    self.variables_names[col] = old_name + "+"
                    self.variables_names.insert(col+1, old_name+ "-")
                    col += 2
                    
                else: 
                    A_ub_bar[:A_ub.shape[0], col] = A_ub[:,i]
                    A_eq_bar[:A_eb.shape[0], col] = A_eq[:,i]
                    if u[i] < np.inf: 
                        A_ub_bar[A_ub.shape[0]+lin, col] = 1
                        A_ub_bar[A_ub.shape[0]+lin+1, col] = -1 
                        b_ub_var[A_ub.shape[0]+lin] = u[i]
                        b_ub_var[A_ub.shape[0]+lin+1] = -l[i]
                        lin += 2
                    col += 1
        return (c_bar, A_ub_bar, A_eq_bar, b_ub_bar, b_eq)

    def _add_slack_variables(self, A_ub, b_ub):
        
        if A_ub.shape[0] == 0 or b_ub.shape[0] == 0: 
            return (np.empty(shape=(0,0)), A_ub, b_ub)
        
        b_ub = b_ub.reshape(-1,1)
        assert A_ub.shape[0] == b_ub.shape[0]
        
        slack_var = np.eye(A_ub.shape[0])
        return (slack_var, A_ub, b_ub)
        
    def _force_b_positive(self, slack_var, A, b): 

        negative_const = np.where(b < 0)[0]
        A[negative_const] = -A[negative_const]
        slack_var[negative_const] = slack_var[negative_const]
        b = abs(b)
        return slack_var, A, b
    
    def _add_artificial_variables(self, slack, A, c, b): 
        
        slack_bar = np.eyes(slack.shape[0], slack.shape[0]+sum(slack.sum(axis = 1) == -1))
        additional_variables = ["s"+str(i) for i in slack_bar.shape[1]]
        additional_variables.extend(self.variables_names)
        self.variables_names = list(additional_variables)
        
        slack_bar[np.where(slack==-1)[0], slack.shape[0]:] = -1.
        
        self.table = np.zeros(slack_bar.shape[0]+1, slack_bar.shape[1]+A.shape[1]+1)
        self.table[:-1, :] = np.hstack([slack_bar, A, b])
        self.table[-1, slack_bar.shape[1]:-1] = c
        
        art_variables = np.where(slack.sum(axis = 1) <= 0)
        for i in art_variables:
            self.variables_names[i] = "a" + self.variables_names[i][1]
        
        return art_variables
        
    def _phase1(self, table, var): 
        
        new_objective = table[self.art_variables, :].sum(axis=0)
        table = np.vstack([table, new_objective])
        table[-1, art_variables] = 0.0
        table[-1,-1] = 0.0
        table, var = self._iteration(table, var)
        
        if stable[-1,-1] < new_objective[-1]:
            raise Exception("The problem is infeasible.")
        else: 
            for i in range(table.shape[0]-2): 
                if var[i][0] == "a": 
                    for e, v in enumerate(var): 
                        if v[i][0] != "a" && table[i,e] != 0:
                            a_ie = table[i,e]
                            norm_line_i = tb[i,:]/a_ie
                            table = table - np.outer(table[:,e], table[i,:])/a_ie
                            table[i,:] = norm_line_i
                            var[i], var[e] = var[e], var[i]
                            table[:,[i,e]] = table[:,[e,i]]                            
                            break
                    table = np.vstack([table[:i,:], table[i+1:,:]])
            for e, v in enumerate(var): 
                if v[0] == "a": 
                    table = np.hstack([table[:,:e], table[:,e+1:]])
                    var.remove(v)
            table = table[:-1,:]
            
        return table, var                 
        
    def _phase2(self, table, var):
        
        return self._iteration(table, var)
    
    def _iteration(self,tb, var): 
        
        while sum(tb[-1,:-1] > 0) > 0: 
            
            s = np.argmax(tb[-1,:-1])
            if sum(tb[:-1,s] > 0) == 0: 
                print("The primal problem is unbounded!")
                return np.nan
            
            positive_a = np.where(tb[:-1,s] > 0)[0]
            r = positive_a[(tb[:-1,-1]/tb[:-1,s])[positive_a].argmin()]

            a_rs = tb[r,s]
            norm_line_r = tb[r,:]/a_rs
            tb = tb - np.outer(tb[:,s], tb[r,:])/a_rs
            tb[r,:] = norm_line_r
            
            # Change of variables
            var[r], var[s] = var[s], var[r]
            tb[:,[r,s]] = tb[:,[s,r]]
            
        return (tb, var)
        
    def optimize(self):
        
        tb = np.array(self.table)
        var = list(self.variables_names)
        
        tb, var = self._phase1(tb, var)
        tb, var = self._phase2(tb, var)
    
        print("The program has finished successively!")
        
        self.variables_names = var
        self.program_optimized = tb
        
    def _results(self, table, var): 
        pass
        